# Wall effects on aggregate dynamics

In [19]:
import numpy as np
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt
import GravityMachineTrack
from cycler import cycler
%matplotlib notebook


In [12]:
# Load the Rotation dynamics fitting results

data_folder = 'C:/Users/Deepak/Dropbox/ActiveMassTransport_Vorticella_SinkingAggregates/RotationalAnalysis/FinalAnalysis/RotationalAnalysis_Fitting_Results'
files_list = os.listdir(data_folder)

path_list_1 = []
path_list_2 = []
track_files_1 = []
track_files_2 = []
for file in files_list:
    
    df = pd.read_csv(os.path.join(data_folder, file))
    
    track_file = df['track file'][0]
    
    if(track_file.startswith('H')):
        path_list_1.append(track_file)
    else:
        path_list_2.append(track_file)    


common_path_1 = os.path.commonpath(path_list_1)
common_path_2 = os.path.commonpath(path_list_2)

print(common_path_1)
print(common_path_2)

print(path_list_1)


H:\2019 Monterey Trip\Vorticella_GM
D:\Vorticella_GravityMachine\2019_08_23
['H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track8/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track10/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track10/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track10/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track10/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track10/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track11/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track10/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_22/Track3/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_22/Track3/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_22/track8/track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_22/track9/track000.cs

In [15]:
track_files_1 = []
track_files_2 = []

track_files_final_1 = {}
track_files_final_2 = {}

# Define a new common-path to find the files
common_path_1_new = 'E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019'
common_path_2_new = 'E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019/2019_08_23'

for file in path_list_1:
    track_files_final_1[file] = os.path.join(common_path_1_new, os.path.relpath(file, start = common_path_1))
    
for file in path_list_2:
    track_files_final_2[file] = os.path.join(common_path_2_new, os.path.relpath(file, start = common_path_2))
    
print(track_files_final_1)
print(track_files_final_2)






        
            
   


{'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track8/track000.csv': 'E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\\2019_08_21_Afternoon\\Track8\\track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track10/track000.csv': 'E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\\2019_08_21_Afternoon\\Track10\\track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_21_Afternoon/Track11/track000.csv': 'E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\\2019_08_21_Afternoon\\Track11\\track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_22/Track3/track000.csv': 'E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\\2019_08_22\\Track3\\track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_22/track8/track000.csv': 'E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\\2019_08_22\\track8\\track000.csv', 'H:/2019 Monterey Trip/Vorticella_GM/2019_08_22/track9/track000.csv': 'E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\\2019_08_22

In [17]:
# Save the new track locations in the CSV files
for file in files_list:
    
    df = pd.read_csv(os.path.join(data_folder, file))
    
    track_file = df['track file'][0]
    df['new track file'] = None
    if(track_file in track_files_final_1.keys()):
        df['new track file'][0] = track_files_final_1[track_file]
    else:
        df['new track file'][0] = track_files_final_2[track_file]
    
    df.to_csv(os.path.join(data_folder, file))

C:\Users\Deepak\Anaconda3\envs\gm_py3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Deepak\Anaconda3\envs\gm_py3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [25]:
# For each file, find the distance to nearest wall at the given time points
import scipy.interpolate as interpolate

for file in files_list:
    
    df = pd.read_csv(os.path.join(data_folder, file))
    
    track_file =  df['new track file'][0]
    
    time_stamps = np.array(df['Time (s)'])
    
    folder, *rest = os.path.split(track_file)
    
    # Load the wall-location data
    
    df_wall = pd.read_csv(os.path.join(folder, 'WallLocations.csv'))
    
    # Load the actual track data
    track = GravityMachineTrack.gravMachineTrack(trackFile = track_file, findDims = False)
    
    func_X = interpolate.interp1d(track.df['Time'], track.df[track.Xobj_name], kind = 'linear')
    func_Y = interpolate.interp1d(track.df['Time'], track.df[track.Yobj_name], kind = 'linear')
    
    
    X_pos = func_X(time_stamps)
    Y_pos = func_Y(time_stamps)
    
    print(X_pos)
    print(Y_pos)
    # Store the aggregate location at the particular times as well as wall-locations for the track
    
    df['X pos (mm)'] = None
    df['Y pos (mm)'] = None
    df['Front wall (mm)'] = None
    df['Back wall (mm)'] = None
    
    df['X pos (mm)'] = X_pos
    df['Y pos (mm)'] = Y_pos
    df['Front wall (mm)'] = np.repeat(df_wall['Front (mm)'][0], len(time_stamps))
    df['Back wall (mm)'] = np.repeat(df_wall['Back (mm)'][0], len(time_stamps))
    
    df.to_csv(os.path.join(data_folder, file))

# Save that info along with the rotational dynamics data

Path : E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\2019_08_21_Afternoon\Track8
['Time', 'Xobj', 'Yobj', 'Xobj_image', 'Zobj', 'ThetaWheel', 'ZobjWheel', 'Manual Tracking', 'Image name', 'Focus Measure', 'Liquid Lens Phase', 'Liquid Lens Freq', 'Liquid Lens Ampl', 'Liquid Lens maxGain', 'Y FM maximum', 'Light Experiment', 'LED_Intensity', 'Lens position']
0
3158
Track duration : 84.52801632881163
Sampling frequency: 37.34856367289347
Smoothing window: 0.2677479136167616
Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[-5.674      -5.66083454 -5.66425215 -5.652      -5.64302838 -5.62260819
 -5.61549237 -5.5997678  -5.58354031 -5.5697267  -5.554      -5.54420622
 -5.53177844 -5.526      -5.50975135 -5.49537603 -5.47895808 -5.46319779
 -5.44052848 -5.41303867 -5.40127376 -5.38281655 -5.35862403 -5.346
 -5.328      -5.306      -5.27769591 -5.24958972 -5.21815205 -5.18584795
 -5.148      -5.10616937 -5.07       -5.02241625 -4.9855

Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[2.67417188 2.72       2.77418186 2.82197678 2.86932329 2.91426214
 2.95184795 2.9867483  3.03491228 3.07127012 3.10364214 3.13162801
 3.15065177 3.178      3.20298698 3.2227794  3.25184795 3.27184795
 3.29278078 3.32355703 3.34461484 3.368      3.3951556  3.416
 3.43815529]
[0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338
 0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338 0.338
 0.338]
Path : E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\2019_08_22\track8
['Time', 'Xobj', 'Yobj', 'Xobj_image', 'Zobj', 'ThetaWheel', 'ZobjWheel', 'Manual Tracking', 'Image name', 'Focus Measure', 'Liquid Lens Phase', 'Liquid Lens Freq', 'Liquid Lens Ampl', 'Liquid Lens maxGain', 'Y FM maximum', 'Light Experiment', 'LED_Intensity', 'Lens position']
0
1598
Track duration : 38.04849982261657
Sampling frequency: 41.97274550758294
Smoothing window: 0.2382498423457

Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[1.40520967 1.4295631  1.45381868 1.48184795 1.48493165 1.50143902
 1.51798091 1.53517357 1.53235042 1.57136953 1.58841625 1.59769591
 1.62056793 1.66724858 1.68048462 1.71060627 1.71272428]
[2.53       2.53       2.53       2.53       2.53       2.53
 2.53       2.53       2.53       2.53       2.53       2.53
 2.53       2.594465   2.68822748 2.78412064 2.842     ]
Path : E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\2019_08_22\Track5
['Time', 'Xobj', 'Yobj', 'Xobj_image', 'Zobj', 'ThetaWheel', 'ZobjWheel', 'Manual Tracking', 'Image name', 'Focus Measure', 'Liquid Lens Phase', 'Liquid Lens Freq', 'Liquid Lens Ampl', 'Liquid Lens maxGain', 'Y FM maximum', 'Light Experiment', 'LED_Intensity', 'Lens position']
0
40064
Track duration : 942.3334379196167
Sampling frequency: 42.51467515410131
Smoothing window: 0.23521289916372132
Computed Image height, Image width: 1080, 1920
0.00119

Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[12.27166617 12.286      12.31       12.33584795 12.35987442]
[1.866      1.866      1.866      1.91582043 1.87187442]
Path : E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\2019_08_22\Track5
['Time', 'Xobj', 'Yobj', 'Xobj_image', 'Zobj', 'ThetaWheel', 'ZobjWheel', 'Manual Tracking', 'Image name', 'Focus Measure', 'Liquid Lens Phase', 'Liquid Lens Freq', 'Liquid Lens Ampl', 'Liquid Lens maxGain', 'Y FM maximum', 'Light Experiment', 'LED_Intensity', 'Lens position']
0
40064
Track duration : 942.3334379196167
Sampling frequency: 42.51467515410131
Smoothing window: 0.23521289916372132
Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[12.384      12.374      12.36796053 12.364      12.36       12.356
 12.354      12.352      12.35       12.348      12.344      12.342
 12.33474866]
[1.464 1.464 1.464 1.464 1.464 1.464 1.464 1.464 1.464

Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[11.05186606 11.04245959 11.06615205 11.07       11.058      11.05048318
 11.058      11.06279039 11.06249141 11.06180187 11.064      11.05843433
 11.04019002 11.04277793 11.03015205 11.01001412 11.02850075 11.01422112
 11.014     ]
[0.822 0.822 0.822 0.822 0.822 0.822 0.822 0.822 0.822 0.822 0.822 0.822
 0.822 0.822 0.822 0.822 0.822 0.822 0.822]
Path : E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019\2019_08_22_afterdinner\track2
['Time', 'Xobj', 'Yobj', 'Xobj_image', 'Zobj', 'ThetaWheel', 'ZobjWheel', 'Manual Tracking', 'Image name', 'Focus Measure', 'Liquid Lens Phase', 'Liquid Lens Freq', 'Liquid Lens Ampl', 'Liquid Lens maxGain', 'Y FM maximum', 'Light Experiment', 'LED_Intensity', 'Lens position']
0
7144
Track duration : 282.49356269836426
Sampling frequency: 25.285531931313493
Smoothing window: 0.3954830781161477
Computed Image height, Image width: 1080, 1920
0.0011942675159

Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[0.88413118 0.89224822 0.90573466 0.9228482  0.9282507  0.944
 0.95737084]
[-1.678 -1.678 -1.678 -1.678 -1.678 -1.678 -1.678]
Path : E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019/2019_08_23\track1
['Time', 'Xobj', 'Yobj', 'Xobj_image', 'Zobj', 'ThetaWheel', 'ZobjWheel', 'Manual Tracking', 'Image name', 'Focus Measure', 'Liquid Lens Phase', 'Liquid Lens Freq', 'Liquid Lens Ampl', 'Liquid Lens maxGain', 'Y FM maximum', 'Light Experiment', 'LED_Intensity', 'Lens position']
0
32729
Track duration : 811.9831230640411
Sampling frequency: 40.30625646072541
Smoothing window: 0.24810044092643643
Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[1.028      1.03       1.0402357  1.048      1.052      1.058
 1.0646104  1.07       1.07067328 1.078      1.078     ]
[-1.874 -1.874 -1.874 -1.874 -1.874 -1.874 -1.874 -1.874 -1.874 -1.874
 -1.874

Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[-0.12054726 -0.09898306 -0.07232274 -0.05397656 -0.02015205]
[-1.182 -1.182 -1.182 -1.182 -1.182]
Path : E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019/2019_08_23\track1
['Time', 'Xobj', 'Yobj', 'Xobj_image', 'Zobj', 'ThetaWheel', 'ZobjWheel', 'Manual Tracking', 'Image name', 'Focus Measure', 'Liquid Lens Phase', 'Liquid Lens Freq', 'Liquid Lens Ampl', 'Liquid Lens maxGain', 'Y FM maximum', 'Light Experiment', 'LED_Intensity', 'Lens position']
0
32729
Track duration : 811.9831230640411
Sampling frequency: 40.30625646072541
Smoothing window: 0.24810044092643643
Computed Image height, Image width: 1080, 1920
0.001194267515923567
Pixel per mm : 837.3333333333333
[9.046      9.06       9.07879556 9.09584795 9.106      9.126     ]
[-2.188 -2.188 -2.188 -2.188 -2.188 -2.188]
Path : E:/GravityMachine_Deepak/Vorticella_GM_Monterey_2019/2019_08_23\track1
['Time', 'Xobj', 'Yobj', 'Xobj_ima